In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd drive/MyDrive

/content/drive/MyDrive


In [3]:
# install dependencies: (use cu111 because colab has CUDA 11.1)
!pip install torch==1.11.0+cu111 torchvision==0.12.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html

# Install mmdetection
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

!pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.11.0+cu111 (from versions: 1.4.0, 1.4.0+cpu, 1.4.0+cu100, 1.4.0+cu92, 1.5.0, 1.5.0+cpu, 1.5.0+cu101, 1.5.0+cu92, 1.5.1, 1.5.1+cpu, 1.5.1+cu101, 1.5.1+cu92, 1.6.0, 1.6.0+cpu, 1.6.0+cu101, 1.6.0+cu92, 1.7.0, 1.7.0+cpu, 1.7.0+cu101, 1.7.0+cu110, 1.7.0+cu92, 1.7.1, 1.7.1+cpu, 1.7.1+cu101, 1.7.1+cu110, 1.7.1+cu92, 1.7.1+rocm3.7, 1.7.1+rocm3.8, 1.8.0, 1.8.0+cpu, 1.8.0+cu101, 1.8.0+cu111, 1.8.0+rocm3.10, 1.8.0+rocm4.0.1, 1.8.1, 1.8.1+cpu, 1.8.1+cu101, 1.8.1+cu102, 1.8.1+cu111, 1.8.1+rocm3.10, 1.8.1+rocm4.0.1, 1.9.0, 1.9.0+cpu, 1.9.0+cu102, 1.9.0+cu111, 1.9.0+rocm4.0.1, 1.9.0+rocm4.1, 1.9.0+rocm4.2, 1.9.1, 1.9.1+cpu, 1.9.1+cu102, 1.9.1+cu111, 1.9.1+rocm4.0.1, 1.9.1+rocm4.1, 1.9.1+rocm4.2, 1.10.0, 1.10.0+cpu, 1.10.0+cu102, 1.10.0+cu111, 1.10.0+cu113, 1.10.0+r

In [4]:
!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_caffe_fpn_mstrain_3x_coco/faster_rcnn_r50_caffe_fpn_mstrain_3x_coco_20210526_095054-1f77628b.pth \
      -O checkpoints/faster_rcnn_r50_caffe_fpn_mstrain_3x_coco_20210526_095054-1f77628b.pth

--2022-12-15 08:48:51--  https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_caffe_fpn_mstrain_3x_coco/faster_rcnn_r50_caffe_fpn_mstrain_3x_coco_20210526_095054-1f77628b.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.254.186.246
Connecting to download.openmmlab.com (download.openmmlab.com)|47.254.186.246|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 167291982 (160M) [application/octet-stream]
Saving to: ‘checkpoints/faster_rcnn_r50_caffe_fpn_mstrain_3x_coco_20210526_095054-1f77628b.pth’

checkpoints/faster_ 100%[===================>] 159.54M  7.36MB/s    in 19s     

2022-12-15 08:49:11 (8.22 MB/s) - ‘checkpoints/faster_rcnn_r50_caffe_fpn_mstrain_3x_coco_20210526_095054-1f77628b.pth’ saved [167291982/167291982]



In [5]:
import os
%cd /content/drive/MyDrive/archive/train_zip/train
file_lst = os.listdir()
img_lst = [i[:-4] for i in file_lst if i[-1]=='g']
with open('train.txt', 'w') as f:
  for i in img_lst:
    f.write(f'{i}\n')

/content/drive/MyDrive/archive/train_zip/train


In [6]:
%cd /content/drive/MyDrive/archive/test_zip/test
file_lst = os.listdir()
img_lst = [i[:-4] for i in file_lst if i[-1]=='g']
with open('test.txt', 'w') as f:
  for i in img_lst:
    f.write(f'{i}\n')

/content/drive/MyDrive/archive/test_zip/test


In [7]:
%cd /content/drive/MyDrive/mmdetection
import copy
import os.path as osp

import mmcv
import numpy as np
import xml.etree.ElementTree as ET

from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset

/content/drive/MyDrive/mmdetection


/usr/local/lib/python3.8/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [8]:
@DATASETS.register_module()
class FruitDataset(CustomDataset):

  CLASSES = ('apple', 'banana', 'orange')

  def load_annotations(self, ann_file):
    cat2label = {k:i for i, k in enumerate(self.CLASSES)}

    image_list = mmcv.list_from_file(self.ann_file)

    data_infos = []
    for image_id in image_list:
      filename = f'{self.img_prefix}/{image_id}.jpg'
      image = mmcv.imread(filename)
      height, width = image.shape[:2]

      data_info = {
          'filename': f'{image_id}.jpg',
          'width': width,
          'height': height
      }

      # load annotations
      ann_tree = ET.parse(f'{self.img_prefix}/{image_id}.xml')
      ann_root = ann_tree.getroot()

      gt_bboxes = []
      gt_labels = []
      for ann_info in ann_root.findall('object'):
        gt_labels.append(cat2label[ann_info.find('name').text])
        bb = ann_info.find('bndbox')
        bb = [bb.find('xmin').text, bb.find('ymin').text,
            bb.find('xmax').text, bb.find('ymax').text]
        bb = [int(i) for i in bb]
        gt_bboxes.append(bb)
      
      data_anno = {
          'bboxes': np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
          'labels': np.array(gt_labels, dtype=np.long),
          'bboxes_ignore': np.array([], dtype=np.float32).reshape(-1, 4),
          'labels_ignore': np.array([], dtype=np.long)
      }
      data_info.update(ann=data_anno)
      data_infos.append(data_info)
    return data_infos


In [9]:
from mmcv import Config
cfg = Config.fromfile('configs/faster_rcnn/faster_rcnn_r50_caffe_fpn_mstrain_1x_coco.py')
from mmdet.apis import set_random_seed

cfg.dataset_type = 'FruitDataset'
cfg.data_root = '../archive'

cfg.data.test.type = 'FruitDataset'
cfg.data.test.data_root = '../archive'
cfg.data.test.ann_file = 'test_zip/test/test.txt'
cfg.data.test.img_prefix = 'test_zip/test'

cfg.data.train.type = 'FruitDataset'
cfg.data.train.data_root = '../archive'
cfg.data.train.ann_file = 'train_zip/train/train.txt'
cfg.data.train.img_prefix = 'train_zip/train'

cfg.data.val.type = 'FruitDataset'
cfg.data.val.data_root = '../archive'
cfg.data.val.ann_file = 'test_zip/test/test.txt'
cfg.data.val.img_prefix = 'test_zip/test'

cfg.model.roi_head.bbox_head.num_classes = 3

cfg.load_from = 'checkpoints/faster_rcnn_r50_caffe_fpn_mstrain_3x_coco_20210526_095054-1f77628b.pth'

cfg.work_dir = './tutorial_exps'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 0.02 / 8
cfg.lr_config.warmup = None
cfg.log_config.interval = 10

# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'mAP'
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 12
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 12

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.device = 'cuda'
cfg.gpu_ids = range(1)

# We can also use tensorboard to log the training process
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='TensorboardLoggerHook')]

# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='FasterRCNN',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=False),
        norm_eval=True,
        style='caffe',
        init_cfg=dict(
            type='Pretrained',
            checkpoint='open-mmlab://detectron2/resnet50_caffe')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='Cro

In [18]:
%cd /content/drive/MyDrive/mmdetection
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector


# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(cfg.model)
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

/content/drive/MyDrive/mmdetection


<ipython-input-8-80b3ac5cd4eb>:39: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  'labels': np.array(gt_labels, dtype=np.long),
<ipython-input-8-80b3ac5cd4eb>:41: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 60/60, 8.6 task/s, elapsed: 7s, ETA:     0s
---------------iou_thr: 0.5---------------


2022-12-15 09:49:43,570 - mmdet - INFO - 
+--------+-----+------+--------+-------+
| class  | gts | dets | recall | ap    |
+--------+-----+------+--------+-------+
| apple  | 35  | 47   | 0.971  | 0.938 |
| banana | 40  | 60   | 0.925  | 0.896 |
| orange | 42  | 55   | 0.976  | 0.967 |
+--------+-----+------+--------+-------+
| mAP    |     |      |        | 0.934 |
+--------+-----+------+--------+-------+
2022-12-15 09:49:43,820 - mmdet - INFO - Epoch(val) [12][60]	AP50: 0.9340, mAP: 0.9339


In [19]:
from mmdet.apis.inference import show_result_pyplot, inference_detector
%cd /content/drive/MyDrive/archive/test_zip/test
with open('test.txt', 'r') as f:
  img_lst = [i[:-1] for i in f.readlines()]

for img_name in img_lst:
  img = mmcv.imread(f'{img_name}.jpg')
  model.cfg = cfg
  res = inference_detector(model, img)
  t = show_result_pyplot(model, img, res, out_file=f'../res/{img_name}_res.jpg')

/content/drive/MyDrive/archive/test_zip/test
